# 概要

![jupyter](./思维导图.png)

- 基于上述思维导图，案例分析顺序为‘车金融项目重要指标概览’、‘渠道名单数量分析’、‘渠道名单质量分析’、‘通话时长分析’、‘通话次数分析’、‘风控周期波动性分析’、‘直属门店分析’；
- 此篇记录详尽记录代码细节及分析思路；


# 一、数据加载及清洗

## 数据加载&预览

In [5]:
import pandas as pd # 数据处理包
import numpy as np # 向量、矩阵运算包
import matplotlib.pyplot as plt # 作图包

df = pd.read_excel(r'E:\迅雷下载\借呗京东数据每日追踪明细-新_20190731.xlsx') # 加载mysql数据到内存
pd.set_option('display.max_columns', None) # 显示所有列
df.head() #预览数据前五行

,渠道来源,ID,是否申请验车,所在省,所在市,所在地区,省,市,姓名,脱敏手机号,借款人uid,初始额度,日利率,申请时间,失效时间,分配时间,坐席名字,坐席uid,小组名称,部门名称,订单状态,当前状态,邀约结果,派单时间,派单省份,派单城市,派单门店,门店退回时间,是否按揭,预约到店时间,到店时间,门店提交初审时间,终审通过时间,终审额度,提款成功时间,放款金额,放款门店,数据累计通次,数据累计接通通次,数据累计有效通次,数据累计通时,当天通次,当天接通通次,当天有效通次,当天通时,近3日通次,近3日接通通次,近3日通时,近7日通次,近7日接通通次,近7日通时,近15日通次,近15日接通通次,近15日通时,本月累计通次,本月累计接通通次,本月累计通时
0,借呗,201907030711260045A,是,四川省,成都市,龙泉驿区,四川省,成都市,段岳朋,136****2373,23107879,58000.0,0.0,2019-07-31 22:58:17.0,2019-08-02 14:50:03.0,NaN,NaN,0,NaN,NaN,其他,默认,NaN,NaN,NaN,NaN,NaN,NaN,未知,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,借呗,201907140702858290A,是,NaN,NaN,NaN,贵州省,铜仁市,谭红通,175****1995,19584558,85000.0,0.0,2019-07-31 22:56:35.0,2019-08-13 22:34:32.0,2019-02-16 01:46:22.0,俞嘉杰,14889211,二组-借呗客服,客服,其他,未联系,未接通,NaN,NaN,NaN,NaN,NaN,未知,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,3.0,1.0,1.0,51.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,借呗,201907130717726257A,是,新疆维吾尔自治区,伊犁哈萨克自治州,伊宁市,新疆维吾尔自治区,伊犁哈萨克自治州,袁新兰,180****2029,23106315,68000.0,0.0,2019-07-31 18:44:46.0,2019-08-12 11:26:20.0,NaN,NaN,0,NaN,NaN,其他,默认,NaN,NaN,NaN,NaN,NaN,NaN,未知,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,借呗,201907070700827415A,是,浙江省,杭州市,余杭区,浙江省,杭州市,徐晓波,139****3948,23106163,94000.0,0.0,2019-07-31 18:29:01.0,2019-08-06 21:06:33.0,NaN,NaN,0,NaN,NaN,其他,默认,NaN,NaN,NaN,NaN,NaN,NaN,未知,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,借呗,201907030688664001A,是,福建省,泉州市,晋江市,福建省,泉州市,王焕发,136****7555,22527292,159000.0,0.0,2019-07-31 18:23:59.0,2019-08-02 16:42:26.0,NaN,NaN,0,NaN,NaN,其他,默认,NaN,NaN,NaN,NaN,NaN,NaN,未知,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,3.0,3.0,3.0,283.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,3.0,283.0


##  数据清洗

- 预览可知部分列对当前分析无意义，需删除；

In [6]:
df.shape # 查看原始行列数

(231281, 57)

In [7]:
drop_columns = [ '渠道来源','是否申请验车','脱敏手机号','日利率','失效时间','是否按揭','订单状态','当前状态','邀约结果','派单时间','派单省份','派单城市','派单门店','预约到店时间','到店时间','提款成功时间','放款门店','当天通次', '当天接通通次', '当天有效通次', '当天通时', '近3日通次', '近3日接通通次', '近3日通时','近7日通次', '近7日接通通次', '近7日通时', '近15日通次', '近15日接通通次', '近15日通时', '本月累计通次', '本月累计接通通次', '本月累计通时']
df.drop(drop_columns,inplace=True,axis=1) # 删除列；变量drop_columns为需要删除字段的列表（list）
df.shape # 查看修改后行列数

(231281, 24)

- 预览可知数据集有缺失、空值，需处理；

In [8]:
df.replace('',np.nan,inplace=True) # 将空格值替换成np.nan统一进行空值处理；
df.replace(r'\s+', np.nan, regex=True,inplace=True)# 将空格值替换成np.nan统一进行空值处理；参数regex=True启用正则；

注：基于数据集理解可把所有空值简单替换成数值0；具体数据集具体分析空值处理

In [9]:
df = df.fillna(0) # 将所有空值替换为数值0；

In [10]:
df.rename(columns={'数据累计通时  ': '数据累计通时'},inplace=True) # 修改列名；列名含有空格

- 预览可知数据集有异常值，需处理；

In [11]:
df[((df['放款金额']<20000)&(df['放款金额']>0))|((df['数据累计通次']<0)|(df['数据累计接通通次']<0)|(df['数据累计接通通次']<0)|(df['数据累计有效通次']<0)|(df['数据累计通时']<0))]
# 检测异常值

,ID,所在省,所在市,所在地区,省,市,姓名,借款人uid,初始额度,申请时间,分配时间,坐席名字,坐席uid,小组名称,部门名称,门店退回时间,门店提交初审时间,终审通过时间,终审额度,放款金额,数据累计通次,数据累计接通通次,数据累计有效通次,数据累计通时
181228,201901010447544001A,上海市,上海市,浦东新区,江苏省,苏州市,高九龙,18197932,81000.0,0.0,0.0,谷俊远,14522914,二组-借呗客服,客服,0.0,0.0,0.0,0.0,0.0,4.0,1.0,1.0,-173.0


In [12]:
df.loc[df['ID']=='201901010447544001A','数据累计通时'] = 45 # 将异常值赋值为正常值

- 防止数据集'ID'列有重复值，需处理；

In [13]:
df.shape # 查看原始行列数

(231281, 24)

In [14]:
df.drop_duplicates(subset='ID') # 删除含有重复值的行，基于‘ID’字段；参数subset指定基于哪个字段删除重复行
df.shape # 查看修改后行列数

(231281, 24)

- 重设索引，索引为Pandas默认，需重设为‘ID’字段为索引；注：索引唯一性，需先去重，空、缺失值处理；

In [15]:
df.set_index('ID',inplace=True) # 更改索引

- 防止str类型值中含有空格，需处理

In [19]:
str_columns = ['所在省', '所在市', '所在地区', '省', '市', '姓名', '坐席名字','小组名称', '部门名称']
# 需要去除str类型值含有

for i in str_columns: # fro循环遍历去除str类型值中含有的空格
    df['%s'%(i)].str.strip()

- 查看列的类型，合理更正。

In [20]:
df.info() # 查看数据集基本信息

<class 'pandas.core.frame.DataFrame'>
Index: 231281 entries, 201907030711260045A to TEST_SN_20180702_001
Data columns (total 23 columns):
所在省         231281 non-null object
所在市         231281 non-null object
所在地区        231281 non-null object
省           231281 non-null object
市           231281 non-null object
姓名          231281 non-null object
借款人uid      231281 non-null int64
初始额度        231281 non-null float64
申请时间        231281 non-null float64
分配时间        231281 non-null float64
坐席名字        231281 non-null object
坐席uid       231281 non-null int64
小组名称        231281 non-null object
部门名称        231281 non-null object
门店退回时间      231281 non-null float64
门店提交初审时间    231281 non-null float64
终审通过时间      231281 non-null float64
终审额度        231281 non-null float64
放款金额        231281 non-null float64
数据累计通次      231281 non-null float64
数据累计接通通次    231281 non-null float64
数据累计有效通次    231281 non-null float64
数据累计通时      231281 non-null float64
dtypes: float64(12), int64(2), object(9)
memory

In [21]:
date_colums = ['申请时间','分配时间','门店退回时间','门店提交初审时间','终审通过时间'] # 数据类型需要转换为时间类型的列

for i in date_colums: # for循环 批量修改
    df['%s'%(i)] = pd.to_datetime(df['%s'%(i)], format ='%Y-%m-%d') # 数据类型修改为时间类型
    
df['数据累计通时'] = df['数据累计通时'].astype('float64') # 数据类型需要转换为浮点数的列

# 二、数据探索